<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Linear Time Series Modeling

_Author: Matt Brems, Matthew Garton, Charles Rice_

---

### Learning Objectives 
_By the end of the lesson, students should be able to:_
- Instantiate and manipulate datetime objects
- Define forecasting.
- Define and identify trend and seasonality in time series data.
- Define and calculate autocorrelation manually.
- Generate and interpret a seasonal decomposition plot.
- Generate and interpret an autocorrelation plot.
- Generate and interpret a partial autocorrelation plot.
- Properly fit, generate predictions from, and evaluate a linear time series model.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# This will allow us to avoid a FutureWarning when plotting.
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

<a id="the-datetime-library"></a>
## The Datetime Library
---

Python's `datetime` library is great for dealing with time-related data, and Pandas has incorporated this library into its own datetime series and objects.

In this lesson, we'll review these data types and learn a little more about each of them:

- Datetime objects.
- Datetime series.
- Timestamps.
- `timedelta()`.


<a id="datetime-object"></a>
## Datetime Objects
---

Below, we read in the `datetime` library which we can use to create a `datetime` object by entering in the different components of the date as arguments. 

It should be noted that there are separate `date` and `time` modules _within_ `datetime`. We're going to skip over those and go straight to the big kahuna, but they follow similar syntax.

In [ ]:
from datetime import datetime

The pieces of the date are accessible via the objects attributes:

In [ ]:
# A:


## `strftime()` and `strptime()`
​
A `datetime` object has two string methods that can be of use: `strftime()` and `strptime()`.  

`strftime()` (think string _from_ time) formats a datetime object into a string following a particular format.

`strptime()` (think string _parsed to_ time) parses a string into a datetime object following a particular format.

Formatting syntax follows the C implementation, and requires some playing to get right. Here is the [full list of formatting options](https://docs.python.org/3.6/library/datetime.html?highlight=strftime#strftime-and-strptime-behavior).

<a id="timedelta"></a>
## `timedelta()`
---

Suppose we want to add time to a date or subtract time from it. Maybe we are using time as an index and want to get everything that happened a week before a specific observation.

We can use a `timedelta` object to shift a datetime object. Here's an example:

In [ ]:
# Import timedelta() from the datetime library.
from datetime import timedelta

# Timedeltas represent time as an amount rather than as a fixed position.
offset = timedelta(days=2, seconds=23, microseconds=32,
                   milliseconds=99, minutes=42, hours=11, weeks=2)

The timedelta() has attributes that allow us to extract values from it.
Per the documentation, only days, seconds, microseconds are stored. 
Mls, mins, hours, weeks are converted to:

* A millisecond is converted to 1000 microseconds.
* A minute is converted to 60 seconds.
* An hour is converted to 3600 seconds.
* A week is converted to 7 days.

Thus, this works:

In [ ]:
print('offset days', offset.days)
print('offset seconds', offset.seconds)
print('offset microseconds', offset.microseconds)

This does not:

In [ ]:
print('offset hours', offset.hours)
# print('offset weeks', offset.weeks)
# print('offset milliseconds', offset.milliseconds)

Datetime's .now() function will give you the datetime object of this very moment. Like so:

The current time is particularly useful when using `timedelta()`:

In [ ]:
# A:


> _Note: The largest value a `timedelta()` can hold is `days`. For instance, you can't say you want your offset to be two years, 44 days, and 12 hours; you have to convert those years to days._

You can read more in the `timedelta()` category [here](https://docs.python.org/3.6/library/datetime.html).


<img src="https://64.media.tumblr.com/f4731e18b4faba6bf86d0ed7fd9a88dd/tumblr_ngoznzXkSw1qlvq41o5_250.gifv">

## Practice: Load the UFO Reports Data
---

We can practice using datetime functions and objects with the UFO reports data.

In [ ]:
# Get the data from the interwebz
ufo = pd.read_csv('http://bit.ly/uforeports')

In [ ]:
# Check the data.


If you've worked with Excel or other spreadsheet software, you know that computers can have a mind of their own when it comes to reading human-input timestamps. We can't take it on faith that data we get are formatted correctly, _especially_ when it comes to dates and times.

In [ ]:
# What kind of data does our `Time` column contain?


It's an 'O' which _might_ be fine because datetime objects are objects in pandas and python. But they're always called out as some form of `datetime`.

In [ ]:
# Let's make sure.


<a id="pandas-pddatetime"></a>
## Pandas and Datetime
---

Using Pandas, we can convert columns of data from string objects into date objects with the `pd.to_datetime()` function.

**Optional Check**: Look at the Pandas `to_datetime` function, and answer the following questions:

1. What does this function do?
2. What does the `format` argument do?
3. What does the `errors` argument do?

In [ ]:
# Overwrite the original Time column with one that has been converted to a datetime series.

# Check the data.


<a id="the-dt-attribute"></a>
### The `.dt` Attribute

Pandas' datetime columns have a `.dt` attribute that allows you to access attributes specific to dates. For example:

```python
ufo.Time.dt.day
ufo.Time.dt.month
ufo.Time.dt.year
ufo.Time.dt.days_in_month
```

[And there are many more!](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#api-series-dt)

In [ ]:
# Let's check one out.


### Inferring `datetime`

We can also tell pandas to parse (or try to parse) datetime objects when we're reading in a `.csv`.

If we take a look at `pd.read_csv`, a few parameters standout:
* parse_dates
* infer_datetime_format

In [ ]:
ufo_2 = pd.read_csv('http://bit.ly/uforeports',
                    parse_dates=[4],
                   infer_datetime_format=True)

`Parse_dates` takes three types of values: boolean, lists, and dictionaries.
* Boolean (default=False) True will attempt to parse the index as a datetime
* List [1,2,3] will attempt to parse the several columns into datetime objects
* Nested List [[1,2,3]] will attempt to combine the several columns into one datetime column

In [ ]:
# Check our data again.


In [ ]:
# Confirm that the datetime format is correct.


## Timestamps
---

Timestamps are useful objects for comparisons. You can create a timestamp object with the `pd.to_datetime()` function and a string specifying the date. These objects are especially helpful when you need to perform logical filtering with dates.

In [ ]:
# For example:

# Use that timestamp for a comparison.


That is all by way of a _very_ brief introduction to everything that can be done using datetime objects in python and pandas, and how we can use them to manipulate our data. Let's move on to forecasting.

## Forecasting

Our main focus during time series week is not to conduct inference, but to generate forecasts for future values of one or more variables.

<details><summary>What do you think forecasting is?</summary>

- Forecasting is predicting the future as accurately as possible.
</details>

**We should first determine whether or not we want to forecast.**
- Our model's performance will depend on many things:
    - How much data is available.
    - How well we understand factors that contribute to the thing we want to predict. (a.k.a. how predictable is the thing we want to predict!)
    - Whether forecasts can affect what we want to forecast.

### Notation

Imagine a dataset that looks like this:

|  **time**  | $Y_t$ | $Y_{t-1}$ | $Y_{t-2}$ | $Y_{t-3}$ | $X_{t}$ |
|-------|-------|-----------|-----------|-----------|---------|
| **1** | $y_1$ | NaN       | NaN       | NaN       | $x_1$   |
| **2** | $y_2$ | $y_1$     | NaN       | NaN       | $x_2$   |
| **3** | $y_3$ | $y_2$     | $y_1$     | NaN       | $x_3$   |
| **4** | $y_4$ | $y_3$     | $y_2$     | $y_1$     | $x_4$   |

We'll fit models that can help us forecast time series.
- **The thing we want to predict is** $Y_t$.
- We may use $X_t$ to predict $Y_t$.
- We may use $Y_{t-1}$, the value of $Y$ at time $t-1$, to predict $Y_t$.

In [ ]:
# Instantiate dataframe with index.
df =

# Set seed.
np.random.seed(42)

# Generate data.
data = [52.4]
for i in range(1,df.shape[0]):
    data.append(data[i-1] + np.random.normal())
    
# Put data in DataFrame.
df['co2'] = data

## Describing the Behavior of a Time Series through Plots

When attempting to forecast time series data, it is critical for us to describe the behavior of the data. (*This is going to tell us how to model our data!*)

We generally plot the data, then identify whether or not a few things are exhibited: trends, seasonality, and autocorrelation.

The most common type of plot is called a **time plot**.

<img src="./images/airlines.png" alt="drawing" width="500"/>

- A time plot is a line plot with time on the horizontal axis and the value of our series on the vertical axis.

To make things easier, we're going to write a function that will generate plots for us.

In [ ]:
# Write function called plot_series that takes in 
# a dataframe, a list of column names to plot, the 
# plot title and the axis labels as arguments,
# then displays the line plot with a figure size
# of 18 horizontal inches by 9 vertical inches.

# Matthew Garton - BOS

def plot_series(df, cols=None, title='Title', xlab=None, ylab=None):
    
    # Set figure size to be (18, 9).
    plt.figure(figsize=(18,9))
    
    # Iterate through each column name.
    for col in cols:
        
        # Generate a line plot of the column name.
        # You only have to specify Y, since our
        # index will be a datetime index.
        plt.plot(df[col])
        
    # Generate title and labels.
    plt.title(title, fontsize=26)
    plt.xlabel(xlab, fontsize=20)
    plt.ylabel(ylab, fontsize=20)
    
    # Enlarge tick marks.
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18);

In [ ]:
plot_series()

<details><summary>How would you describe this plot?</summary>
    
_(Answers will vary.)_

- Across the full range of time, the value of $Y$ increases.
- There is a sharp jump upward around July 2019.
- $Y$ increases and decreases over time, but in irregular patterns.
- The values of $Y_t$ and $Y_{t-1}$ are highly correlated with one another.
</details>

### Trends

In time series data, a **trend** is the long-term increase or decrease in the data.

The two most common types of trends will be **linear (straight line) trends** and **exponential trends**.

<img src="./images/column_1_small.png" alt="drawing" width="550"/>

<details><summary>Is there a trend in the carbon dioxide output? If so, is it increasing or decreasing? Do you think it's a straight line trend or an exponential trend?</summary>

- Yes, there is a trend in the graph. There appears to be a long-term increase over time.
- The trend could be either linear or exponential!
</details>

We'll decide how to model the above shortly.

### Seasonality

In time series data, **seasonality** describes when a time series is affected by factors that take on a **fixed and known frequency**.

Seasonality does not have to explicitly refer to seasons of the year (and from an inclusivity standpoint, different regions of the world have different seasons), but seasonality almost always **will** be tied to some meaning on the calendar!

<details><summary>Can you think of examples of time series data that exhibit seasonality?</summary>
    
- Sales data for many products likely exhibits seasonality. Purchases of heavier coats will probably peak in colder times of the year, which will occur roughly every twelve months.
- Restaurant sales likely exhibits seasonality with a peak every week (usually Friday or Saturday).
- Temperature itself would exhibit seasonality! We expect temperature to fluctuate up and down in a relatively consistent pattern every twelve months or so.
    - Note that temperature also has an increasing trend... so trends and seasonality can occur together!
</details>

<img src="./images/column_1_small.png" alt="drawing" width="550"/>

<details><summary>Is there seasonality in the carbon dioxide output? If so, about how frequently?</summary>

- No, there does not appear to be seasonality.
- While $Y$ does fluctuate up and down, it does not appear to fluctuate with some fixed and known frequency.
</details>

One tool that _can_ be helpful is the `seasonal_decompose` tool from `statsmodels`. However, we have to be careful!

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
# Decompose time series into trend, seasonal, and residual components.
decomp =

# Plot the decomposed time series.
;

By default, the decomposition will assume there is a linear trend. If you believe there is an exponential trend, you should change the argument `model` to be `multiplicative`, but this will fail with values that are less than or equal to 0.

The plot will **always** pull out a seasonal component... even if there isn't a seasonal component in our data! In short, this is sometimes a helpful tool, but it's not a substitute for plotting your data with a time plot and examining it first.

### Autocorrelation

In time series data, autocorrelation refers to the **correlation of one variable with lagged versions of itself**. (You may also hear the term **serial correlation**.)

You might imagine a dataset that looks like this:

|  **time**  | $Y_t$ | $Y_{t-1}$ | $Y_{t-2}$ | $Y_{t-3}$ | $X_{t}$ |
|-------|-------|-----------|-----------|-----------|---------|
| **1** | $y_1$ | NaN       | NaN       | NaN       | $x_1$   |
| **2** | $y_2$ | $y_1$     | NaN       | NaN       | $x_2$   |
| **3** | $y_3$ | $y_2$     | $y_1$     | NaN       | $x_3$   |
| **4** | $y_4$ | $y_3$     | $y_2$     | $y_1$     | $x_4$   |

In [ ]:
# Generate a dataframe with our CO2 data that
# looks like the above dataframe.

df['co2_lag_1'] =
df['co2_lag_2'] =
df['co2_lag_3'] =

In [ ]:
df.head()

We can calculate the correlation between $Y_t$ and $Y_{t-1}$, which tells us how highly correlated values of $Y$ are with the immediately previous value of $Y$.

<details><summary>BONUS: If you want to see the formula for autocorrelation, check here.</summary>
    
$$
\begin{eqnarray*}
Corr(Y_t, Y_{t-k}) = \frac{Cov(Y_t,Y_{t-k})}{Var(Y_t)Var(Y_{t-k})}
\end{eqnarray*}
$$
</details> 

In [ ]:
# Calculate the correlations among these columns.


<details><summary>Why do you think it's useful to look at the correlation between our $Y$ variable and lagged versions of itself?</summary>
    
- It can inform us which lagged values of $Y$ to put into our model!
</details>

We can also calculate the correlation between $Y_t$ and $Y_{t-2}$, $Y_t$ and $Y_{t-3}$, and so on. But this can be tedious to do by hand, so we generally use a function to generate an **autocorrelation plot** that visualizes this all at once!

In [ ]:
# Import the autocorrelation function (ACF) plot.
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
# Generate the ACF plot on co2 data
# up to 20 time periods.
;

### Reading an Autocorrelation Plot

The **horizontal axis** tells us which lag of $Y_t$ we are comparing to the main time series $Y_t$.

The **vertical axis** tells us the value of the autocorrelation between $Y_t$ and the lagged version, $Y_{t-lag}$.

- The left-most value, $lag = 0$ of the autocorrelation plot is $Corr(Y_t, Y_t)$. 
    - This should always be 1!
- The next value, $lag = 1$ of the autocorrelation plot is $Corr(Y_t, Y_{t-1})$.
- The next value of the autocorrelation plot visualizes $Corr(Y_t, Y_{t-2})$.

The **blue oval** is our confidence interval. If an element is _outside_ the blue area, it is statistically significant.




#### Interpretation

The autocorrelation plot is an important tool for a few reasons:
- **We can use it to detect trends**: When the autocorrelation plot has large and positive values for small lags (e.g. lags 1, 2, 3), this is evidence that a trend exists.
- **We can use it to detect seasonality**: Recall that seasonality means we have fluctuations in our $Y$ variable that occur with a fixed and known frequency. When the autocorrelation plot has larger values for the seasonal lags than other lags, this is evidence that seasonality exists. (Visually, this might look like a scalloped shape.)
- **We can use this plot to inform our modeling choices**, along with a related plot - the partial autocorrelation function plot.

### Partial Autocorrelation

Based on the autocorrelation plot above, we might try to fit a model that looks like this:

$$
\begin{eqnarray*}
Y_t &=& \beta_0 + \beta_1Y_{t-1} + \beta_2Y_{t-2} + \cdots + \beta_{20}Y_{t-20} \\
&=& \beta_0 + \sum_{k=1}^{20}\beta_kY_{t-k}
\end{eqnarray*}
$$



<details><summary>However, this can run into many problems!</summary>

- The more variables we have, the likelier we are to overfit our model to the data.
- $Y_{t-1}$ and $Y_{t-2}$ and $Y_{t-3}$ are all highly correlated with one another, so the inputs to our model in this case aren't independent of one another.
- When we use a variable like $Y_{t-20}$, we effectively decrease our sample size by 20. (If our data are measured year over year, discarding 20 years of data is **a lot of data** to exclude from our model.)
</details>

To avoid this problem, we use the **partial autocorrelation function**.
- The partial autocorrelation is like the autocorrelation in that it checks for the correlation between $Y_t$ and lagged versions of itself.
- However, the partial autocorrelation **controls for all lower-lag autocorrelations**.
    - That is, the partial autocorrelation between $Y_t$ and $Y_{t-2}$ is the correlation between $Y_t$ and $Y_{t-2}$  that has already taken into account the autocorrelation between $Y_t$ and $Y_{t-1}$.
    
    
<details><summary>BONUS: If you want to see the formula for partial autocorrelation, check here.</summary>
    
$$
\begin{eqnarray*}
PartialCorr(Y_t, Y_{t-k}) = \frac{Cov(Y_t,Y_{t-k}|Y_{t-1},Y_{t-2},\ldots,Y_{t-(k-1)})}{Var(Y_t|Y_{t-1},Y_{t-2},\ldots,Y_{t-(k-1)})Var(Y_{t-k}|Y_{t-1},Y_{t-2},\ldots,Y_{t-(k-1)})}
\end{eqnarray*}
$$
</details>     

In [ ]:
# Import the partial autocorrelation function (PACF) plot.
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
# Generate the PACF plot on CO2 data
# up to 20 time periods.
;

### Reading a Partial Autocorrelation Plot

The **horizontal axis** still tells us which lag of $Y_t$ we are comparing to the main time series $Y_t$.

The **vertical axis** tells us the value of the **partial autocorrelation** between $Y_t$ and the lagged version, $Y_{t-lag}$.

The blue band represents a 95% confidence interval. If the value of the partial autocorrelation goes outside the blue band, that means that correlation is statistically significant.

#### Interpretation

The partial autocorrelation plot is an important tool for a few reasons:
- **We can use it to detect seasonality**: Recall that seasonality means we have fluctuations in our $Y$ variable that occur with a fixed and known frequency. When the partial autocorrelation plot has significant values for the seasonal lags, this is evidence that seasonality exists.
- **We can use this plot to inform our modeling choices**: Here, we see that by using $Y_{t-1}$ as a predictor for $Y_t$, we probably won't get much important information if we were to also include $Y_{t-2}$ (or higher order lags) as a predictor.

## Recap

When you look at a new time series, you should:
- generate a time plot.
- generate the autocorrelation plot.
- generate the partial autocorrelation plot.

Use these plots to determine:
- is there a trend?
- is there seasonality?

Once we answer these questions, we can begin to model.

## Practice

On your own, take some time to read in the following data and answer the following questions.
1. Based on the time plot, is there evidence of a trend? If so, describe the trend. How can you tell?
2. Based on the time plot, is there evidence of seasonality? If so, estimate the frequency of the seasonality. How can you tell?
3. Based on the ACF plot, is there evidence of a trend? If so, describe the trend. How can you tell?
4. Based on the ACF and PACF plots, is there evidence of seasonality? If so, estimate the frequency of the seasonality. How can you tell?

In [ ]:
# Read in airline data.
air = pd.read_csv('./data/airline-passengers.csv', parse_dates=['Month'])

# Set the index to be month over month.


In [ ]:
air.head()

In [ ]:
# Generate a time plot of the airline passengers data.


In [ ]:
# Generate a decomposition plot of the airline passengers data.


In [ ]:
# Generate an ACF plot of the airline passengers data with 30 time periods.


In [ ]:
# Generate an PACF plot of the airline passengers data with 30 time periods.


<details><summary>1. Based on the time plot, is there evidence of a trend? If so, describe the trend. How can you tell?</summary>
    
- There is a trend. It seems to be increasing and pretty linear.
</details>

<details><summary>2. Based on the time plot, is there evidence of seasonality? If so, estimate the frequency of the seasonality. How can you tell?</summary>
    
- There is evidence of seasonality. There appear to be peaks in the data about every twelve months. This makes sense, because the number of people flying on planes in a given month may peak when there's more travel (e.g. summer) and it might die down in other months.
- This evidence is strongest in the later years, where from 1954-1961 there's obviously one peak every year. (Note that there are two peaks between 1956 and 1958, two peaks between 1958 and 1960, etc.)
</details>

<details><summary>3. Based on the ACF plot, is there evidence of a trend? If so, describe the trend. How can you tell?</summary>

- Yes, there is a trend. In the ACF plot, the small lag values have large, positive autocorrelations.
</details>

<details><summary>4. Based on the ACF and PACF plots, is there evidence of seasonality? If so, estimate the frequency of the seasonality. How can you tell?</summary>
    
- Yes, there is seasonality. In the ACF plot, there is a "scalloped" shape, where we see ACF values peak roughly every 12 months.
- In the PACF plot, we see significant values every 12 months or so. We also see some positive and some negative significant partial autocorrelations, which usually indicates strong seasonal fluctuations.
</details>

<details><summary>Based on the above plots, we might conclude:</summary>

- The last two values are highly informative in predicting future values.
- There is a linear trend in our data.
- There is seasonality that occurs roughly every twelve months.

Let's fit the following model:
$$
\begin{eqnarray*}
Y_t &=& \beta_0 + \beta_1Y_{t-1} + \beta_2Y_{t-2} + \beta_3Y_{t-12} + \beta_4t
\end{eqnarray*}
$$
</details>

In [ ]:
air.head()

In [ ]:
# Create a column called `lag_1` that lags Passengers by one month.
air['lag_1'] =

# Create a column called `lag_2` that lags Passengers by two months.
air['lag_2'] =

# Create a column called `seasonal_12` that lags Passengers by one year.
air['seasonal_12'] =

# Create a variable called `time` that takes on a value of 0 in January 1949,
# then increases by 1 each month until the end of the dataframe.
air['time'] =

In [ ]:
# Confirm the top of our dataframe looks good.
air.head()

In [ ]:
# Confirm the bottom of our dataframe looks good.


## Train-Test Split

Before building a model, we should split our data up into a training and testing set.

Since our goal with time series models is almost always to forecast values forward in time, the idea with a time series train/test split is to train on earlier data and test/evaluate on later data.

Most commonly, we'll set our:
- training set to be the "first" 67% - 80% of our data timewise.
- test set be the "last" 20% - 33% timewise.

Let's split our dataframe by taking the first 80% of rows for training and the rest for testing.

In [ ]:
# Generate train/test split.
X_train, X_test, y_train, y_test = train_test_split(air.drop(columns = 'Passengers'),
                                                    air['Passengers'],
                                                    test_size = 0.2)

In [ ]:
# Check shape to confirm we did this properly.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## Fit a linear model.

In [ ]:
# Import statsmodels.
import statsmodels.api as sm

In [ ]:
# Before fitting a model in statsmodels, what do we need
# to do? (Hint: Think intercept.)




# Confirm.


In [ ]:
# statsmodels won't be able to handle missing values.




# This way we subset y_train to keep only indices from X_train.

In [ ]:
# Remember that, in statsmodels, we pass our data 
# in when we instantiate the model!

lm =

In [ ]:
# Then we fit our model.
lm_results =

In [ ]:
# Display our summary!
print(lm_results.summary())

In [ ]:
# Generate predicted test values.


In [ ]:
# Import R2 score and MSE.
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
# Calculate R2 score.
r2_score()

In [ ]:
# Calculate RMSE.
mean_squared_error() ** 0.5

In [ ]:
# Let's plot our predictions! 

# Set figure size.
plt.figure(figsize=(20,10))

# Plot training data.
plt.plot(y_train.index, y_train.values, color = 'blue')

# Plot testing data.
plt.plot(y_test.index, y_test.values, color = 'orange')

# Plot predicted test values.
plt.plot(lm_results.predict(X_test), color = 'green')

# Set label.
plt.title(label = 'Forecasting Airline Passengers 1958-1961', fontsize=24)

# Resize tick marks.
plt.xticks(fontsize=20)
plt.yticks(fontsize=20);

### Learning Objectives 
_By the end of the lesson, students should be able to:_
- Define forecasting.
- Define and identify trend and seasonality in time series data.
- Define and calculate autocorrelation manually.
- Generate and interpret a seasonal decomposition plot.
- Generate and interpret an autocorrelation plot.
- Generate and interpret a partial autocorrelation plot.
- Properly fit, generate predictions from, and evaluate a linear time series model.